통계왕의 Public 93.17 -> Private 13등 솔루션    

이 노트북은 2048개의 트레인 데이터로  
클래스의 소프트맥스 확률이 0.9이상 것만 추출해서  
이를 바탕으로 테스트에 대한 예측 값을 훈련에 재사용하는 기법인  
수도 라벨링을 활용하였습니다.  
  
  
유의하실 점은 수도라벨링을 위한 기초 데이터 생성 시, 어그멘테이션을 적용하지 않았으며,  
SpinalVGG만 활용하여 테스트 데이터를 예측하였고, 이 예측값을  
모델 훈련에 재사용하였습니다.  
  
이후 SpinalVGG와 VGG를 활용하여 5폴드 앙상블을 하였습니다.  
이 결과 기존 2048개의 예측의 Public Score가 87.74에 불과하였는데  
Pesudo Labelling을 통하여 Public Score가 93.17까지 상승하였습니다.  

또한 10개의 모델 앙상블을 통해 Private Score는 92.365로, Shake Down을 방지하였습니다.

필요 모듈 로드

In [17]:
import torch
import torchvision
import torch.nn as nn
import math
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import os
from sklearn.model_selection import GroupKFold, StratifiedKFold
from PIL import Image
import pandas as pd

Pesudo Labelling을 적용 하지 않을 경우에는 data_transform에서 ToTensor()와 Normalize만 적용한다

In [22]:
data_transforms = {
    'train': transforms.Compose([

        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),

        transforms.Normalize([0.5], [0.5])
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
}

데이터로드 정의

In [26]:
class train_dataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.n_idx = len(df)
        self.train_data = self.df.loc[:,'0':].astype(float).values.reshape(-1,28,28)
        self.label = self.df['digit'].astype(int).values
    def __len__(self):
        return self.n_idx
    
    def __getitem__(self, idx):
        train = self.train_data[idx]
        label = torch.from_numpy(np.array(self.label[idx]))
        train = Image.fromarray(train)
        train = data_transforms['train'](train)
        return train, label
    
class test_dataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.n_idx = len(df)
        self.test_data = self.df.loc[:,'0':].astype(float).values.reshape(-1,28,28)
    def __len__(self):
        return self.n_idx
    
    def __getitem__(self, idx):
        test = self.test_data[idx]
        test = Image.fromarray(test)
        test = data_transforms['val'](test)
        return test

In [56]:
df = pd.read_csv("./emnist/data/test.csv")

In [57]:
df

,id,letter,0,1,2,3,4,5,6,7,...,774,775,776,777,778,779,780,781,782,783
0,2049,L,0,4,0,2,4,2,3,1,...,2,0,4,2,2,4,3,4,1,4
1,2050,C,4,1,4,0,1,1,0,2,...,0,3,2,4,2,4,2,2,1,2
2,2051,S,0,4,0,1,3,2,3,0,...,1,3,2,0,3,2,3,0,1,4
3,2052,K,2,1,3,3,3,4,3,0,...,3,0,3,2,4,1,0,4,4,4
4,2053,W,1,0,1,1,2,2,1,4,...,4,3,1,4,0,2,1,2,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20475,22524,P,1,2,1,1,0,0,0,2,...,0,1,3,0,3,0,4,3,1,4
20476,22525,S,4,1,1,4,0,0,1,1,...,1,3,1,0,0,1,3,1,2,0
20477,22526,B,4,2,1,3,2,1,3,0,...,3,2,3,4,1,0,3,3,1,1
20478,22527,K,1,1,2,3,4,0,4,3,...,2,0,0,4,3,3,3,4,4,2


두개의 네트워크 정의  
  VGG, SpinalVGG  
  두 모델을 앙상블 할 예정임

In [ ]:
# Two Network
class VGG(nn.Module):  
    def two_conv_pool(self, in_channels, f1, f2):
        s = nn.Sequential(
            nn.Conv2d(in_channels, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.Conv2d(f1, f2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        for m in s.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        return s
    
    def three_conv_pool(self,in_channels, f1, f2, f3):
        s = nn.Sequential(
            nn.Conv2d(in_channels, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.Conv2d(f1, f2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(inplace=True),
            nn.Conv2d(f2, f3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        for m in s.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        return s
        
    
    def __init__(self, num_classes=10):
        super(VGG, self).__init__()
        self.l1 = self.two_conv_pool(1, 64, 64)
        self.l2 = self.two_conv_pool(64, 128, 128)
        self.l3 = self.three_conv_pool(128, 256, 256, 256)
        self.l4 = self.three_conv_pool(256, 256, 256, 256)
        
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, num_classes),
        )
    
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

In [ ]:
class SpinalVGG(nn.Module):  
    def two_conv_pool(self, in_channels, f1, f2):
        s = nn.Sequential(
            nn.Conv2d(in_channels, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.Conv2d(f1, f2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        for m in s.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        return s
    
    def three_conv_pool(self,in_channels, f1, f2, f3):
        s = nn.Sequential(
            nn.Conv2d(in_channels, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.Conv2d(f1, f2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(inplace=True),
            nn.Conv2d(f2, f3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        for m in s.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        return s
        
    
    def __init__(self, num_classes=10):
        super(SpinalVGG, self).__init__()
        self.l1 = self.two_conv_pool(1, 64, 64)
        self.l2 = self.two_conv_pool(64, 128, 128)
        self.l3 = self.three_conv_pool(128, 256, 256, 256)
        self.l4 = self.three_conv_pool(256, 256, 256, 256)
        
        
        self.fc_spinal_layer1 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(Half_width, layer_width),
            nn.BatchNorm1d(layer_width), nn.ReLU(inplace=True),)
        self.fc_spinal_layer2 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(Half_width+layer_width, layer_width),
            nn.BatchNorm1d(layer_width), nn.ReLU(inplace=True),)
        self.fc_spinal_layer3 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(Half_width+layer_width, layer_width),
            nn.BatchNorm1d(layer_width), nn.ReLU(inplace=True),)
        self.fc_spinal_layer4 = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(Half_width+layer_width, layer_width),
            nn.BatchNorm1d(layer_width), nn.ReLU(inplace=True),)
        self.fc_out = nn.Sequential(
            nn.Dropout(p = 0.5), nn.Linear(layer_width*4, num_classes),)
        
    
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = x.view(x.size(0), -1)
        
        x1 = self.fc_spinal_layer1(x[:, 0:Half_width])
        x2 = self.fc_spinal_layer2(torch.cat([ x[:,Half_width:2*Half_width], x1], dim=1))
        x3 = self.fc_spinal_layer3(torch.cat([ x[:,0:Half_width], x2], dim=1))
        x4 = self.fc_spinal_layer4(torch.cat([ x[:,Half_width:2*Half_width], x3], dim=1))
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.cat([x, x3], dim=1)
        x = torch.cat([x, x4], dim=1)
        
        x = self.fc_out(x)

        return F.log_softmax(x, dim=1)

훈련 시작  
만약 수도라벨링을 하지 않는다면 pesudo_test_data를 로드할 필요가 없습니다

In [28]:
num_epochs = 200
root_path = "./emnist"
batch_size_train = 100
batch_size_test = 2048
learning_rate = 0.005
momentum = 0.5
log_interval = 500
device = 'cuda' 
Half_width =128
layer_width =128

train_data = pd.read_csv("./emnist/data/train.csv")
gkf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (Tidx, Vidx) in enumerate(gkf.split(train_data, train_data['digit'])):
    
# 2048개의 데이터 중에서 해당 클래스의 확률이 0.9 이상만 추출한 파일 활용
    train_data = pd.read_csv("./emnist/data/train.csv")
    pesudo_test_data = pd.read_csv("./emnist/data/pesudo_train.csv")
    
    val_dataset = train_dataset(train_data.iloc[Vidx])
    train_data = train_data.iloc[Tidx]
    # 수도라벨링 데이터와 트레인 데이터의 CONCAT
    train_data = pd.concat([train_data, pesudo_test_data], axis=0).reset_index(drop=True)
    dataset = train_dataset(train_data)
    

    
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size_train, shuffle=True)

    test_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size_test, shuffle=False)

    def update_lr(optimizer, lr):    
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

# Train the model
    total_step = len(train_loader)
    curr_lr1 = learning_rate

    curr_lr2 = learning_rate



    model1 = VGG().to(device)

    model2 = SpinalVGG().to(device)



# Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer1 = torch.optim.Adam(model1.parameters(), lr=learning_rate)
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=learning_rate) 
  
# Train the model
    total_step = len(train_loader)

    best_accuracy1 = 0
    best_accuracy2 =0
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.long)

        # Forward pass
            outputs = model1(images)
            loss1 = criterion(outputs, labels)

        # Backward and optimize
            optimizer1.zero_grad()
            loss1.backward()
            optimizer1.step()
        
            outputs = model2(images)
            loss2 = criterion(outputs, labels)

        # Backward and optimize
            optimizer2.zero_grad()
            loss2.backward()
            optimizer2.step()

            if i == 499:
                print ("Ordinary Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                       .format(epoch+1, num_epochs, i+1, total_step, loss1.item()))
                print ("Spinal Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                       .format(epoch+1, num_epochs, i+1, total_step, loss2.item()))


        
    # Test the model
        model1.eval()
        model2.eval()
        with torch.no_grad():
            correct1 = 0
            total1 = 0
            correct2 = 0
            total2 = 0
            for images, labels in test_loader:
                images = images.to(device, dtype=torch.float)
                labels = labels.to(device, dtype=torch.long)
            
            
                outputs = model1(images)
                _, predicted = torch.max(outputs.data, 1)
                total1 += labels.size(0)
                correct1 += (predicted == labels).sum().item()
            
                outputs = model2(images)
                _, predicted = torch.max(outputs.data, 1)
                total2 += labels.size(0)
                correct2 += (predicted == labels).sum().item()
    
        
            if best_accuracy1>= correct1 / total1:
                curr_lr1 = learning_rate*np.asscalar(pow(np.random.rand(1),3))
                update_lr(optimizer1, curr_lr1)
                print('Test Accuracy of NN: {} % Best: {} %'.format(100 * correct1 / total1, 100*best_accuracy1))
            
            
                with open(os.path.join(root_path, f"vggmodel_fold_{fold}.pt"), "wb") as f:
                  torch.save(
                      {
                          "state": model1.state_dict(),
                      },
                      f,
                    )

            else:
                best_accuracy1 = correct1 / total1
                net_opt1 = model1
                print('Test Accuracy of NN: {} % (improvement)'.format(100 * correct1 / total1))
            
            if best_accuracy2>= correct2 / total2:
                curr_lr2 = learning_rate*np.asscalar(pow(np.random.rand(1),3))
                update_lr(optimizer2, curr_lr2)
                print('Test Accuracy of SpinalNet: {} % Best: {} %'.format(100 * correct2 / total2, 100*best_accuracy2))
            
                with open(os.path.join(root_path, f"spinal_fold_{fold}.pt"), "wb") as f:
                  torch.save(
                    {
                        "state": model2.state_dict(),
                    },
                    f,
                  )
            else:
                best_accuracy2 = correct2 / total2
                net_opt2 = model2
                print('Test Accuracy of SpinalNet: {} % (improvement)'.format(100 * correct2 / total2))

        
            
            model1.train()
            model2.train()

          id  digit letter  0  1  2  3  4  5  6  ...  774  775  776  777  778  \
0       2049      6      L  0  4  0  2  4  2  3  ...    2    0    4    2    2   
1       2050      9      C  4  1  4  0  1  1  0  ...    0    3    2    4    2   
2       2051      8      S  0  4  0  1  3  2  3  ...    1    3    2    0    3   
3       2052      0      K  2  1  3  3  3  4  3  ...    3    0    3    2    4   
4       2053      3      W  1  0  1  1  2  2  1  ...    4    3    1    4    0   
...      ...    ...    ... .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...   
14633  22523      4      P  1  0  2  3  2  1  2  ...    0    1    2    3    4   
14634  22524      4      P  1  2  1  1  0  0  0  ...    0    1    3    0    3   
14635  22525      1      S  4  1  1  4  0  0  1  ...    1    3    1    0    0   
14636  22526      6      B  4  2  1  3  2  1  3  ...    3    2    3    4    1   
14637  22528      0      S  2  1  0  3  0  3  3  ...    0    3    0    1    4   

       779  780  781  782  

c:\users\gerard\.conda\envs\pytorch15\lib\site-packages\ipykernel_launcher.py:306: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
c:\users\gerard\.conda\envs\pytorch15\lib\site-packages\ipykernel_launcher.py:287: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


Test Accuracy of NN: 66.82926829268293 % Best: 67.07317073170732 %
Test Accuracy of SpinalNet: 76.34146341463415 % (improvement)
Test Accuracy of NN: 80.73170731707317 % (improvement)
Test Accuracy of SpinalNet: 77.3170731707317 % (improvement)
Test Accuracy of NN: 78.29268292682927 % Best: 80.73170731707317 %
Test Accuracy of SpinalNet: 75.85365853658537 % Best: 77.3170731707317 %
Test Accuracy of NN: 85.60975609756098 % (improvement)
Test Accuracy of SpinalNet: 80.73170731707317 % (improvement)
Test Accuracy of NN: 85.1219512195122 % Best: 85.60975609756098 %
Test Accuracy of SpinalNet: 80.0 % Best: 80.73170731707317 %
Test Accuracy of NN: 85.85365853658537 % (improvement)
Test Accuracy of SpinalNet: 73.90243902439025 % Best: 80.73170731707317 %
Test Accuracy of NN: 85.1219512195122 % Best: 85.85365853658537 %
Test Accuracy of SpinalNet: 81.95121951219512 % (improvement)
Test Accuracy of NN: 85.36585365853658 % Best: 85.85365853658537 %
Test Accuracy of SpinalNet: 82.4390243902439 % 

c:\users\gerard\.conda\envs\pytorch15\lib\site-packages\ipykernel_launcher.py:287: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


Test Accuracy of NN: 71.95121951219512 % Best: 75.1219512195122 %
Test Accuracy of SpinalNet: 80.0 % (improvement)
Test Accuracy of NN: 50.97560975609756 % Best: 75.1219512195122 %
Test Accuracy of SpinalNet: 58.292682926829265 % Best: 80.0 %


...

Test Accuracy of NN: 85.36585365853658 % (improvement)
Test Accuracy of SpinalNet: 85.85365853658537 % (improvement)
Test Accuracy of NN: 84.8780487804878 % Best: 85.36585365853658 %
Test Accuracy of SpinalNet: 83.90243902439025 % Best: 85.85365853658537 %
Test Accuracy of NN: 85.1219512195122 % Best: 85.36585365853658 %
Test Accuracy of SpinalNet: 87.5609756097561 % (improvement)
Test Accuracy of NN: 85.85365853658537 % (improvement)
Test Accuracy of SpinalNet: 88.78048780487805 % (improvement)
Test Accuracy of NN: 85.85365853658537 % Best: 85.85365853658537 %
Test Accuracy of SpinalNet: 89.02439024390245 % (improvement)
Test Accuracy of NN: 84.14634146341463 % Best: 85.85365853658537 %
Test Accuracy of SpinalNet: 88.78048780487805 % Best: 89.02439024390245 %
Test Accuracy of NN: 82.6829268292683 % Best: 85.85365853658537 %
Test Accuracy of SpinalNet: 84.8780487804878 % Best: 89.02439024390245 %
Test Accuracy of NN: 87.3170731707317 % (improvement)
Test Accuracy of SpinalNet: 87.80487

c:\users\gerard\.conda\envs\pytorch15\lib\site-packages\ipykernel_launcher.py:306: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
c:\users\gerard\.conda\envs\pytorch15\lib\site-packages\ipykernel_launcher.py:287: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


Test Accuracy of NN: 67.97066014669926 % Best: 80.92909535452323 %
Test Accuracy of SpinalNet: 78.239608801956 % (improvement)
Test Accuracy of NN: 79.46210268948656 % Best: 80.92909535452323 %
Test Accuracy of SpinalNet: 71.88264058679707 % Best: 78.239608801956 %
Test Accuracy of NN: 86.55256723716381 % (improvement)
Test Accuracy of SpinalNet: 88.01955990220048 % (improvement)
Test Accuracy of NN: 87.28606356968216 % (improvement)
Test Accuracy of SpinalNet: 85.81907090464547 % Best: 88.01955990220048 %
Test Accuracy of NN: 89.24205378973105 % (improvement)
Test Accuracy of SpinalNet: 85.57457212713936 % Best: 88.01955990220048 %
Test Accuracy of NN: 88.75305623471883 % Best: 89.24205378973105 %
Test Accuracy of SpinalNet: 87.53056234718827 % Best: 88.01955990220048 %
Test Accuracy of NN: 81.17359413202934 % Best: 89.24205378973105 %
Test Accuracy of SpinalNet: 87.28606356968216 % Best: 88.01955990220048 %
Test Accuracy of NN: 88.99755501222494 % Best: 89.24205378973105 %
Test Accur

c:\users\gerard\.conda\envs\pytorch15\lib\site-packages\ipykernel_launcher.py:306: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
c:\users\gerard\.conda\envs\pytorch15\lib\site-packages\ipykernel_launcher.py:287: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead


Test Accuracy of NN: 67.23716381418093 % Best: 68.21515892420538 %
Test Accuracy of SpinalNet: 81.41809290953545 % (improvement)
Test Accuracy of NN: 84.59657701711491 % (improvement)
Test Accuracy of SpinalNet: 79.95110024449878 % Best: 81.41809290953546 %
Test Accuracy of NN: 84.84107579462103 % (improvement)
Test Accuracy of SpinalNet: 86.79706601466992 % (improvement)
Test Accuracy of NN: 86.0635696821516 % (improvement)
Test Accuracy of SpinalNet: 87.28606356968216 % (improvement)
Test Accuracy of NN: 84.59657701711491 % Best: 86.06356968215158 %
Test Accuracy of SpinalNet: 83.61858190709046 % Best: 87.28606356968214 %
Test Accuracy of NN: 86.55256723716381 % (improvement)
Test Accuracy of SpinalNet: 84.84107579462103 % Best: 87.28606356968214 %
Test Accuracy of NN: 85.81907090464547 % Best: 86.55256723716381 %
Test Accuracy of SpinalNet: 74.5721271393643 % Best: 87.28606356968214 %
Test Accuracy of NN: 85.81907090464547 % Best: 86.55256723716381 %
Test Accuracy of SpinalNet: 79.2

In [49]:
test_file = test_dataset(df)
test_loader = torch.utils.data.DataLoader(test_file, batch_size=batch_size_test, shuffle=False)

예측하기  
Spinal VGG, VGG의 Ensemble 

In [92]:
gkf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_true = train_data['digit'].values
oof_preds = []
train_data = pd.read_csv("./emnist/data/train.csv")
for fold in range(5):
    oof_pred = []
    
    dataset = train_dataset(train_data)    
    test_loader = torch.utils.data.DataLoader(test_file, batch_size=batch_size_test, shuffle=False)

    with open(os.path.join(root_path, f"spinal_fold_{fold}.pt"), "rb") as f:
        saved_model = torch.load(f)
          
    model2 = SpinalVGG()
    model2.load_state_dict(saved_model["state"])
    
    model2.cuda()
    model2.eval()
    
    with torch.no_grad():
        for images in test_loader:
            images = images.to(device, dtype=torch.float)
            
            outputs = model2(images)
            oof_pred.append(outputs.data.cpu().detach().numpy())
    oof_pred = np.concatenate(oof_pred, axis=0)
    oof_preds.append(oof_pred)

In [93]:
for fold in range(5):
    oof_pred = []
    test_loader = torch.utils.data.DataLoader(test_file, batch_size=batch_size_test, shuffle=False)
    with open(os.path.join(root_path, f"vggmodel_fold_{fold}.pt"), "rb") as f:
        saved_model = torch.load(f)
          
    model1 = VGG()
    model1.load_state_dict(saved_model["state"])
    
    model1.cuda()
    model1.eval()
    
    with torch.no_grad():
        for images in test_loader:
            images = images.to(device, dtype=torch.float)
            
            outputs = model1(images)

            oof_pred.append(outputs.data.cpu().detach().numpy())
    oof_pred = np.concatenate(oof_pred, axis=0)
    oof_preds.append(oof_pred)

0-5 Spinal VGG  
6-10 VGG  
모든 폴드의 값을 합한 다음에 10으로 나눔

In [94]:
# 0~5 : SpinalVGG 6~10 : VGG
total_oof_preds = oof_preds[0]+oof_preds[1]+oof_preds[2]+oof_preds[3]+oof_preds[4]+ oof_preds[5]+oof_preds[6]+oof_preds[7]+oof_preds[8]+oof_preds[9]
total_oof_preds = total_oof_preds/10
total_oof_preds = np.argmax(total_oof_preds, axis=1)

제출 파일 작성  
수도라벨링 없이 예측 -> Public 87.74  
With 수도라벨링 -> Public 93.137

In [100]:
submission = pd.read_csv('./emnist/data/submission.csv')
submission['digit'] = total_oof_preds

In [101]:
submission.to_csv('(1ndvggspinawithaug)mnist_spinal_vgg_ensemble_submission.csv', index=False)

In [103]:
submission

,id,digit
0,2049,6
1,2050,9
2,2051,8
3,2052,0
4,2053,3
...,...,...
20475,22524,4
20476,22525,1
20477,22526,6
20478,22527,8
